<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW1/blob/main/BigData_HW1_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: ‌Big Data
<br>
<b>Description:</b> HomeWork 1 | Question 1
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# PySpark Setting

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 27 kB/s 
     |████████████████████████████████| 198 kB 46.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9df427e52c27a2a9797418148a2417c8efb87c0dfb7a027c914477aae84b7f84
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

# Load and Prepare Dataset

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id='1-OCBGBtKoY_PadKHcXDyWxHQ2BS8Nulo'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('hw1.zip')

!unzip hw1.zip

Archive:  hw1.zip
   creating: hw1-files/
   creating: hw1-files/q3/
  inflating: hw1-files/q3/patches.csv  
  inflating: hw1-files/q3/lsh.py     
   creating: hw1-files/q1/
  inflating: hw1-files/q1/dataset1.txt  
   creating: hw1-files/q1/.ipynb_checkpoints/
   creating: hw1-files/q2/
  inflating: hw1-files/q2/games_library.txt  
   creating: hw1-files/.ipynb_checkpoints/


In [ ]:
def convert_string_to_list(string):
  if string=="":
    return []
  else:
    return [int(item) for item in string.split(",")]

In [ ]:
import pandas as pd
df = pd.read_csv("/content/hw1-files/q1/dataset1.txt", sep="\t",names=["id", "friend_id"])

In [ ]:
df = df.fillna("")
df["friend_id"] = df["friend_id"].apply(convert_string_to_list)

In [ ]:
rdd=spark.sparkContext.parallelize(df.to_dict('records'), 2)

# Map-Reduce

In [ ]:
def map_function(item):
  fids = sorted(item["friend_id"])
  output = []
    
  for fid1 in fids:
    counter = {}
    for fid2 in fids:

      if fid1 != fid2:
        counter[fid2] = 1
    
    output.append((fid1, counter))

  return output

In [ ]:
from heapq import merge

def reduce_function(value1, value2):
  merged_ids = list(merge(value1.keys(), value2.keys()))

  merged_value = {}
  for id in merged_ids:
    merged_value[id] = value1.get(id, 0) + value2.get(id, 0)

  return merged_value

In [ ]:
mapped_rdd = rdd.flatMap(map_function)

In [ ]:
reduced_rdd = mapped_rdd.reduceByKey(reduce_function)

In [ ]:
output_rdd = reduced_rdd.collect()

In [ ]:
def suggest_new_friend(id, number_top):
  exist_friend_id = df[df["id"]==id]["friend_id"].values

  for item in output_rdd:
    if item[0]==id:
      counter = item[1]
      break

  sorted_new_friend = sorted(counter, key=counter.get, reverse=True)

  suggestion_list = []

  for new_id in sorted_new_friend:
    if new_id not in exist_friend_id:
      suggestion_list.append(new_id)
      if len(suggestion_list) == number_top:
        break

  return suggestion_list

# Example

In [ ]:
for id in [98, 135, 117, 911, 8804]:
  print(f"id = {id}")
  print(f"Suggestion = {suggest_new_friend(id,10)}\n")

id = 98
Suggestion = [575, 18560, 2554, 13654, 16324, 16350, 30134, 30691, 17, 113]

id = 135
Suggestion = [117, 13792, 33060, 112, 629, 5490, 19091, 19114, 19217, 24568]

id = 117
Suggestion = [135, 34164, 12519, 13793, 15314, 19063, 23507, 23510, 25256, 34140]

id = 911
Suggestion = [913, 24456, 39540, 40560, 919, 30984, 30993, 30995, 30996, 33333]

id = 8804
Suggestion = [34179, 34332, 3230, 8677, 11399, 11400, 13182, 13872, 15207, 29745]

